In [17]:
import graph_tool.all as gt
import numpy as np
import json
import pandas as pd

In [18]:
with open('./../data/java.json') as f:
  data = json.load(f)

link_data = pd.DataFrame(data["links"])
link_data

,from,to,strength
0,org.apache.log4j,org.apache.log4j.spi.Filter,7
1,org.apache.log4j,org.apache.log4j.spi.LoggingEvent,11
2,org.apache.log4j,org.apache.log4j.spi.OptionHandler,3
3,org.apache.log4j,java.util.Collections,4
4,org.apache.log4j,java.util.Enumeration,5
...,...,...,...
7483,org.apache.logging.log4j.web,org.springframework.mock.web.MockServletContext,2
7484,org.apache.logging.log4j.web,javax.servlet.AsyncContext,1
7485,org.apache.logging.log4j.web,javax.servlet.http.HttpServlet,1
7486,org.apache.logging.log4j.web,javax.servlet.http.HttpServletResponse,1


In [24]:
g = gt.Graph()

v_list = {}
v_prop_list = []
v_prop = g.new_vertex_property("string")
for i, row in link_data.iterrows():
  if row['from'] not in v_list:
    v_list[row['from']] = g.add_vertex()
    v_prop[v_list[row['from']]] = row['from']
    v_prop_list.append(row['from'])
  if row['to'] not in v_list:
    v_list[row['to']] = g.add_vertex()
    v_prop[v_list[row['to']]] = row['to']
    v_prop_list.append(row['to'])

for i, row in link_data.iterrows():
  g.add_edge(v_list[row['from']], v_list[row['to']])



{'org.apache.log4j': <Vertex object with index '0' at 0x1c5b00120>,
 'org.apache.log4j.spi.Filter': <Vertex object with index '1' at 0x1c5b00190>,
 'org.apache.log4j.spi.LoggingEvent': <Vertex object with index '2' at 0x1c5b00200>,
 'org.apache.log4j.spi.OptionHandler': <Vertex object with index '3' at 0x1c5b00270>,
 'java.util.Collections': <Vertex object with index '4' at 0x1c5b002e0>,
 'java.util.Enumeration': <Vertex object with index '5' at 0x1c5b00350>,
 'java.util.Map': <Vertex object with index '6' at 0x1c5b003c0>,
 'java.util.ResourceBundle': <Vertex object with index '7' at 0x1c5b00430>,
 'java.util.Vector': <Vertex object with index '8' at 0x1c5b004a0>,
 'java.util.concurrent.ConcurrentMap': <Vertex object with index '9' at 0x1c5b00510>,
 'java.util.stream.Collectors': <Vertex object with index '10' at 0x1c5b00580>,
 'org.apache.log4j.bridge.AppenderAdapter': <Vertex object with index '11' at 0x1c5b005f0>,
 'org.apache.log4j.bridge.AppenderWrapper': <Vertex object with index

# Ordered community structure

グラフ構造が、上流->下流へ流れているという状況においてのコミュニティ抽出アルゴリズム


In [20]:
g, _, _, ew, *_ = gt.condensation_graph(g, g.vertex_index, self_loops=False)

state = gt.NestedBlockState(g, base_type=gt.RankedBlockState, state_args=dict(eweight=ew))

for i in range(100):
    state.multiflip_mcmc_sweep(beta=np.inf, niter=10)

print(state.levels[0])
pos = gt.sfdp_layout(g, cooling_step=0.99, multilevel=False, R=50000, rmap=state.levels[0].get_vertex_order())

for v in g.vertices():
    pos[v][1] *= 2


state.levels[0].draw(pos=pos,
                     edge_pen_width=gt.prop_to_size(ew, 1, 5),
                    output="ordered-communities-java.pdf")

<RankedBlockState object with 2 blocks, 0 upstream, 7488 downstream, and 0 lateral edges, degree-corrected, for graph <Graph object, directed, with 2144 vertices and 7488 edges, at 0x1c450eb30>, at 0x1c450d870>


<VertexPropertyMap object with value type 'vector<double>', for Graph 0x1c450eb30, at 0x1c3974460>

In [27]:
state = gt.minimize_blockmodel_dl(g, state=gt.PPBlockState)

# Now we run 100 sweeps of the MCMC with zero temperature, as a
# refinement. This is often not necessary.

state.multiflip_mcmc_sweep(beta=np.inf, niter=100)

state.draw(
    #pos=g.vp.pos,
    output="assortative-community-structure.svg")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x1c50de770, at 0x1c39747c0>